https://thefiddler.substack.com/p/can-you-throw-the-hammer

**This Week’s Fiddler**

Chris Payne recently introduced me to “TGL” golf, which looks bonkers. Here’s a puzzle that’s inspired by one of the rules from TGL:

You and your opponent are competing in a golf match. On any given hole you play, each of you has a 50 percent chance of winning the hole (and a zero percent chance of tying). That said, scorekeeping in this match is a little different from the norm.

Each hole is worth 1 point. Before starting each hole, either you or your opponent can “throw the hammer.” When the hammer has been thrown, whoever did not throw the hammer must either accept the hammer or reject it. If they accept the hammer, the hole is worth 2 points. If they reject the hammer, they concede the hole and its 1 point to their opponent. Both players can throw the hammer on as many holes as they wish. (Should both players decide to throw the hammer at the exact same time—something that can’t be planned in advance—the hole is worth 2 points.)

The first player to reach 3 points wins the match. Suppose all players always make rational decisions to maximize their own chances of winning.

Good news! You have won the first hole, and now lead 1-0. What is your probability of winning the match?


**This Week’s Extra Credit**

Instead of playing to 3 points, now the first player to 5 points wins the match.

Good news (again)! You have won the first hole, and now lead 1-0. What is your probability of winning the match?

---

---

Um,  I am not sure this hammer does anything. Seems to me that if throwing the hammer hurts player A, then it helps player B. So, at least one of the players will always throw the hammer. Once it is thrown, there is no advantage to conceding, because you just start over in a worse situation. So, I think hammers will always be accepted.

Let's code it all up, and see if that's what really happens.

---

In [44]:
def calc_one_scores_probs(rslt, a_pts, b_pts, win_pts, p_a_wins_hole=0.5):
    ab_pts = (a_pts, b_pts)
    rslt[ab_pts] = {}
    if (a_pts >= win_pts):
        rslt[ab_pts]['p_a_wins_overall'] = 1
    elif (b_pts >= win_pts):
        rslt[ab_pts]['p_a_wins_overall'] = 0
    else:
        # If no hammer is thrown, either A or B gets a point, and we move to the corresponding states
        rslt[ab_pts]['p_a_wins_no_hammer'] = p_a_wins_hole * rslt[(a_pts+1,b_pts)]['p_a_wins_overall'] + (1-p_a_wins_hole) * rslt[(a_pts,b_pts+1)]['p_a_wins_overall']

        # If hammer is thrown and accepted, either A or B gets 2 points, and we move to the corresponding states
        rslt[ab_pts]['p_a_wins_yes_hammer'] = p_a_wins_hole * rslt[(a_pts+2,b_pts)]['p_a_wins_overall'] + (1-p_a_wins_hole) * rslt[(a_pts,b_pts+2)]['p_a_wins_overall']

        # A should accept a hammer thrown by B if accepting improves A's win prob relative to conceding
        rslt[ab_pts]['a_shoud_accept'] = (rslt[ab_pts]['p_a_wins_yes_hammer'] >= rslt[(a_pts,b_pts+1)]['p_a_wins_overall'])

        # B should accept a hammer thrown by A if accepting improves B's win prob relative to conceding
        rslt[ab_pts]['b_shoud_accept'] = (rslt[ab_pts]['p_a_wins_yes_hammer'] <= rslt[(a_pts+1,b_pts)]['p_a_wins_overall'])

        # A's prob of winning when throwing a hammer will be the min of the 2 options, since B will choose to help itself.
        rslt[ab_pts]['p_a_wins__a_threw'] = min((rslt[ab_pts]['p_a_wins_yes_hammer'],  rslt[(a_pts+1,b_pts)]['p_a_wins_overall']))

        # B's prob of winning when throwing a hammer will be the max of the 2 options, since A will choose to help itself.
        rslt[ab_pts]['p_a_wins__b_threw'] = max((rslt[ab_pts]['p_a_wins_yes_hammer'],  rslt[(a_pts,b_pts+1)]['p_a_wins_overall']))

        # A and B should throw if it helps them.
        rslt[ab_pts]['a_should_throw'] = rslt[ab_pts]['p_a_wins__a_threw'] >= rslt[ab_pts]['p_a_wins_no_hammer'] 
        rslt[ab_pts]['b_should_throw'] = rslt[ab_pts]['p_a_wins__b_threw'] <= rslt[ab_pts]['p_a_wins_no_hammer'] 

        rslt[ab_pts]['either_throws_hammer'] = rslt[ab_pts]['a_should_throw'] or rslt[ab_pts]['b_should_throw']
        rslt[ab_pts]['both_throw_hammer'] = rslt[ab_pts]['a_should_throw'] and rslt[ab_pts]['b_should_throw']

        if (rslt[ab_pts]['both_throw_hammer']):
            rslt[ab_pts]['p_a_wins_overall'] = rslt[ab_pts]['p_a_wins_yes_hammer']
        elif (rslt[ab_pts]['either_throws_hammer']):
            if (rslt[ab_pts]['a_should_throw']):
                rslt[ab_pts]['p_a_wins_overall'] = rslt[ab_pts]['p_a_wins__a_threw']
            else:
                rslt[ab_pts]['p_a_wins_overall'] = rslt[ab_pts]['p_a_wins__b_threw']
        else:
            rslt[ab_pts]['p_a_wins_overall'] = rslt[ab_pts]['p_a_wins_no_hammer']


def calc_probs(win_pts, verbose=False, p_a_wins_hole=0.5):
    results = {}
    for total_pts in range(2*win_pts, -1, -1):
        for a_pts in range(total_pts+1):
            b_pts = total_pts - a_pts
            calc_one_scores_probs(results, a_pts, b_pts, win_pts, p_a_wins_hole)
    
    if (verbose):
        for k in results:
            print (k, results[k])
        return
    return results
    
#calc_probs(3, True)

Let use that to solve the puzzles.

In [45]:
def solve_fiddler():
    results = calc_probs(3)
    return results[(1,0)]['p_a_wins_overall']
solve_fiddler()

0.75

In [46]:
def solve_extra_credit():
    results = calc_probs(5)
    return results[(1,0)]['p_a_wins_overall']
solve_extra_credit()

0.6875

----

----

----
And we can see that the hammer effect (or lack thereof) is as expected by examining the calculation steps in detail below.
either_throws_hammer is always True. a/b_should_accept is also always True.

In [47]:
calc_probs(3, True)

(0, 6) {'p_a_wins_overall': 0}
(1, 5) {'p_a_wins_overall': 0}
(2, 4) {'p_a_wins_overall': 0}
(3, 3) {'p_a_wins_overall': 1}
(4, 2) {'p_a_wins_overall': 1}
(5, 1) {'p_a_wins_overall': 1}
(6, 0) {'p_a_wins_overall': 1}
(0, 5) {'p_a_wins_overall': 0}
(1, 4) {'p_a_wins_overall': 0}
(2, 3) {'p_a_wins_overall': 0}
(3, 2) {'p_a_wins_overall': 1}
(4, 1) {'p_a_wins_overall': 1}
(5, 0) {'p_a_wins_overall': 1}
(0, 4) {'p_a_wins_overall': 0}
(1, 3) {'p_a_wins_overall': 0}
(2, 2) {'p_a_wins_no_hammer': 0.5, 'p_a_wins_yes_hammer': 0.5, 'a_shoud_accept': True, 'b_shoud_accept': True, 'p_a_wins__a_threw': 0.5, 'p_a_wins__b_threw': 0.5, 'a_should_throw': True, 'b_should_throw': True, 'either_throws_hammer': True, 'both_throw_hammer': True, 'p_a_wins_overall': 0.5}
(3, 1) {'p_a_wins_overall': 1}
(4, 0) {'p_a_wins_overall': 1}
(0, 3) {'p_a_wins_overall': 0}
(1, 2) {'p_a_wins_no_hammer': 0.25, 'p_a_wins_yes_hammer': 0.5, 'a_shoud_accept': True, 'b_shoud_accept': True, 'p_a_wins__a_threw': 0.5, 'p_a_wins__

In [48]:
calc_probs(5, True)

(0, 10) {'p_a_wins_overall': 0}
(1, 9) {'p_a_wins_overall': 0}
(2, 8) {'p_a_wins_overall': 0}
(3, 7) {'p_a_wins_overall': 0}
(4, 6) {'p_a_wins_overall': 0}
(5, 5) {'p_a_wins_overall': 1}
(6, 4) {'p_a_wins_overall': 1}
(7, 3) {'p_a_wins_overall': 1}
(8, 2) {'p_a_wins_overall': 1}
(9, 1) {'p_a_wins_overall': 1}
(10, 0) {'p_a_wins_overall': 1}
(0, 9) {'p_a_wins_overall': 0}
(1, 8) {'p_a_wins_overall': 0}
(2, 7) {'p_a_wins_overall': 0}
(3, 6) {'p_a_wins_overall': 0}
(4, 5) {'p_a_wins_overall': 0}
(5, 4) {'p_a_wins_overall': 1}
(6, 3) {'p_a_wins_overall': 1}
(7, 2) {'p_a_wins_overall': 1}
(8, 1) {'p_a_wins_overall': 1}
(9, 0) {'p_a_wins_overall': 1}
(0, 8) {'p_a_wins_overall': 0}
(1, 7) {'p_a_wins_overall': 0}
(2, 6) {'p_a_wins_overall': 0}
(3, 5) {'p_a_wins_overall': 0}
(4, 4) {'p_a_wins_no_hammer': 0.5, 'p_a_wins_yes_hammer': 0.5, 'a_shoud_accept': True, 'b_shoud_accept': True, 'p_a_wins__a_threw': 0.5, 'p_a_wins__b_threw': 0.5, 'a_should_throw': True, 'b_should_throw': True, 'either_thro

----

----

----
And trying the suggestion to have asymmetric probabilities from the comments. 
It makes the numbers a bit more interesting, but the hammer situation remains the same, presumably because all rounds have the same probabilities of winning/losing.

In [49]:
calc_probs(3,True,0.25)

(0, 6) {'p_a_wins_overall': 0}
(1, 5) {'p_a_wins_overall': 0}
(2, 4) {'p_a_wins_overall': 0}
(3, 3) {'p_a_wins_overall': 1}
(4, 2) {'p_a_wins_overall': 1}
(5, 1) {'p_a_wins_overall': 1}
(6, 0) {'p_a_wins_overall': 1}
(0, 5) {'p_a_wins_overall': 0}
(1, 4) {'p_a_wins_overall': 0}
(2, 3) {'p_a_wins_overall': 0}
(3, 2) {'p_a_wins_overall': 1}
(4, 1) {'p_a_wins_overall': 1}
(5, 0) {'p_a_wins_overall': 1}
(0, 4) {'p_a_wins_overall': 0}
(1, 3) {'p_a_wins_overall': 0}
(2, 2) {'p_a_wins_no_hammer': 0.25, 'p_a_wins_yes_hammer': 0.25, 'a_shoud_accept': True, 'b_shoud_accept': True, 'p_a_wins__a_threw': 0.25, 'p_a_wins__b_threw': 0.25, 'a_should_throw': True, 'b_should_throw': True, 'either_throws_hammer': True, 'both_throw_hammer': True, 'p_a_wins_overall': 0.25}
(3, 1) {'p_a_wins_overall': 1}
(4, 0) {'p_a_wins_overall': 1}
(0, 3) {'p_a_wins_overall': 0}
(1, 2) {'p_a_wins_no_hammer': 0.0625, 'p_a_wins_yes_hammer': 0.25, 'a_shoud_accept': True, 'b_shoud_accept': True, 'p_a_wins__a_threw': 0.25, 'p